In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
import warnings
warnings.filterwarnings('ignore')

- electricity mix data https://ourworldindata.org/electricity-mix [accessed 24/10/2023]
- total electricity generation https://ourworldindata.org/grapher/electricity-generation [accessed 19/10/2023]

In [3]:
crosswalk = pd.read_excel("../../data/MESSAGE/mapping_file_R10edit.xlsx",sheet_name="main")
capacity_data = pd.read_csv("../_data_process/combined_coal_gas_indicators.csv")

# electricity from different sources
el_share = pd.read_csv("electricity-prod-source-stacked.csv")
el_share = el_share[el_share["Code"].isna()==False]

# total electricity generation
el_gen = pd.read_csv("electricity-generation.csv")
el_gen = el_gen[el_gen["Code"].isna()==False]

In [4]:
generation_data = pd.merge(el_share[["Code","Year","Electricity from solar (TWh)","Electricity from wind (TWh)",
                                                "Electricity from nuclear (TWh)","Electricity from gas (TWh)","Electricity from coal (TWh)"]],el_gen,
                         on=["Code","Year"],how="left")
generation_data = pd.merge(generation_data,crosswalk[["AR6_R10","ISO"]],left_on="Code",right_on="ISO",how="left")
generation_data["Share_Solar_Gen"] = generation_data["Electricity from solar (TWh)"]/generation_data["Electricity generation (TWh)"]
generation_data["Share_Wind_Gen"] = generation_data["Electricity from wind (TWh)"]/generation_data["Electricity generation (TWh)"]
generation_data["Share_Nuclear_Gen"] = generation_data["Electricity from nuclear (TWh)"]/generation_data["Electricity generation (TWh)"]
generation_data["Share_Gas_Gen"] = generation_data["Electricity from gas (TWh)"]/generation_data["Electricity generation (TWh)"]
generation_data["Share_Coal_Gen"] = generation_data["Electricity from coal (TWh)"]/generation_data["Electricity generation (TWh)"]
generation_data = generation_data[generation_data["Electricity generation (TWh)"].isna()==False]
generation_data.set_index("Code").to_csv("combined_generation.csv")
generation_data

,Code,Year,Electricity from solar (TWh),Electricity from wind (TWh),Electricity from nuclear (TWh),Electricity from gas (TWh),Electricity from coal (TWh),Entity,Electricity generation (TWh),AR6_R10,ISO,Share_Solar_Gen,Share_Wind_Gen,Share_Nuclear_Gen,Share_Gas_Gen,Share_Coal_Gen
0,AFG,2000,0.00,0.0,0.0,0.0,0.00,Afghanistan,0.47,R10INDIA+,AFG,0.000000,0.0,0.0,0.0,0.000000
1,AFG,2000,0.00,0.0,0.0,0.0,0.00,Afghanistan,0.47,R10REST_ASIA,AFG,0.000000,0.0,0.0,0.0,0.000000
2,AFG,2001,0.00,0.0,0.0,0.0,0.00,Afghanistan,0.59,R10INDIA+,AFG,0.000000,0.0,0.0,0.0,0.000000
3,AFG,2001,0.00,0.0,0.0,0.0,0.00,Afghanistan,0.59,R10REST_ASIA,AFG,0.000000,0.0,0.0,0.0,0.000000
4,AFG,2002,0.00,0.0,0.0,0.0,0.00,Afghanistan,0.69,R10INDIA+,AFG,0.000000,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5907,ZWE,2017,0.01,0.0,0.0,0.0,3.02,Zimbabwe,7.35,R10AFRICA,ZWE,0.001361,0.0,0.0,0.0,0.410884
5908,ZWE,2018,0.02,0.0,0.0,0.0,3.69,Zimbabwe,9.19,R10AFRICA,ZWE,0.002176,0.0,0.0,0.0,0.401523
5909,ZWE,2019,0.03,0.0,0.0,0.0,3.62,Zimbabwe,8.24,R10AFRICA,ZWE,0.003641,0.0,0.0,0.0,0.439320
5910,ZWE,2020,0.03,0.0,0.0,0.0,3.36,Zimbabwe,7.59,R10AFRICA,ZWE,0.003953,0.0,0.0,0.0,0.442688


In [5]:
combined_data_new = pd.merge(generation_data,capacity_data,
                         right_on=["Country Code","Year"],left_on=["Code","Year"],how="left")
combined_data_stats = pd.DataFrame(data={"Country Code":combined_data_new["Code"].unique()})
combined_data_stats = combined_data_stats.set_index("Country Code")

for cntry in combined_data_new["Code"].unique():
    cntry_data = combined_data_new.query("Code==@cntry")
    index_cntry_data = cntry_data.index
    
    ### coal data ###
    # share of coal in electricity
    max_coal_share = cntry_data["Share_Coal_Gen"].max()
    if math.isnan(max_coal_share) == False:
        if max_coal_share == 0:
            combined_data_stats.loc[cntry,"max_coal_share"] = np.nan
            combined_data_stats.loc[cntry,"max_coal_share_year"] = np.nan
        else:
            combined_data_stats.loc[cntry,"max_coal_share"] = max_coal_share
            combined_data_stats.loc[cntry,"max_coal_share_year"] = cntry_data[cntry_data["Share_Coal_Gen"]==max_coal_share]["Year"].iloc[-1]
    # coal generation 
    max_coal_gen = cntry_data["Electricity from coal (TWh)"].max()
    if math.isnan(max_coal_gen) == False: 
        max_coal_gen_year = cntry_data[cntry_data["Electricity from coal (TWh)"]==max_coal_gen]["Year"].iloc[-1]
        if max_coal_gen_year < cntry_data["Year"].max()-5 and cntry_data["Coal_Peak_MW"].max()>1000:
            index_coal_gen_decline = cntry_data[cntry_data["Year"]>=max_coal_gen_year].index
            combined_data_new.loc[index_coal_gen_decline,"Coal_Gen_Decline_TWh"] = max_coal_gen - combined_data_new.loc[index_coal_gen_decline,"Electricity from coal (TWh)"]
            combined_data_new.loc[index_coal_gen_decline,"Coal_Gen_Decline_Perc"] = combined_data_new.loc[index_coal_gen_decline,"Coal_Gen_Decline_TWh"]/max_coal_gen*100
            combined_data_new.loc[index_coal_gen_decline,"year_since_peak_coal_gen"] = combined_data_new.loc[index_coal_gen_decline,"Year"]-max_coal_gen_year
            combined_data_stats.loc[cntry,"max_coal_gen_year"] = max_coal_gen_year
            combined_data_stats.loc[cntry,"max_coal_gen"] = max_coal_gen
    # coal capacity
    max_coal_cap = cntry_data["Coal_Current_MW"].max()
    if math.isnan(max_coal_cap) == False: 
        max_coal_cap_year = cntry_data[cntry_data["Coal_Current_MW"]==max_coal_cap]["Year"].iloc[-1]
        if max_coal_cap_year < cntry_data["Year"].max()-5 and cntry_data["Coal_Peak_MW"].max()>1000:
            index_coal_cap_decline = cntry_data[cntry_data["Year"]>=max_coal_cap_year].index
            combined_data_new.loc[index_coal_cap_decline,"year_since_peak_coal_cap"] = combined_data_new.loc[index_coal_cap_decline,"Year"]-max_coal_cap_year
            combined_data_stats.loc[cntry,"max_coal_cap_year"] = max_coal_cap_year
            combined_data_stats.loc[cntry,"max_coal_cap"] = max_coal_cap
            
    ### gas data ###
    # share of gas in electricity
    max_gas_share = cntry_data["Share_Gas_Gen"].max()
    if math.isnan(max_gas_share) == False:
        if max_gas_share == 0:
            combined_data_stats.loc[cntry,"max_gas_share"] = np.nan
            combined_data_stats.loc[cntry,"max_gas_share_year"] = np.nan
        else:
            combined_data_stats.loc[cntry,"max_gas_share"] = max_gas_share
            combined_data_stats.loc[cntry,"max_gas_share_year"] = cntry_data[cntry_data["Share_Gas_Gen"]==max_gas_share]["Year"].iloc[-1]
    # gas generation
    max_gas_gen = cntry_data["Electricity from gas (TWh)"].max()
    if math.isnan(max_gas_gen) == False:
        max_gas_gen_year = cntry_data[cntry_data["Electricity from gas (TWh)"]==max_gas_gen]["Year"].iloc[-1]
        if max_gas_gen_year < cntry_data["Year"].max()-5 and cntry_data["Gas_Peak_MW"].max()>1000:
            index_gas_gen_decline = cntry_data[cntry_data["Year"]>=max_gas_gen_year].index
            combined_data_new.loc[index_gas_gen_decline,"Gas_Gen_Decline_TWh"] = max_gas_gen - combined_data_new.loc[index_gas_gen_decline,"Electricity from gas (TWh)"]
            combined_data_new.loc[index_gas_gen_decline,"Gas_Gen_Decline_Perc"] = combined_data_new.loc[index_gas_gen_decline,"Gas_Gen_Decline_TWh"]/max_gas_gen*100
            combined_data_new.loc[index_gas_gen_decline,"year_since_peak_gas_gen"] = combined_data_new.loc[index_gas_gen_decline,"Year"]-max_gas_gen_year
            combined_data_stats.loc[cntry,"max_gas_gen_year"] = max_gas_gen_year
            combined_data_stats.loc[cntry,"max_gas_gen"] = max_gas_gen
    # gas capacity
    max_gas_cap = cntry_data["Gas_Current_MW"].max()
    if math.isnan(max_gas_cap) == False: 
        max_gas_cap_year = cntry_data[cntry_data["Gas_Current_MW"]==max_gas_cap]["Year"].iloc[-1]
        if max_gas_cap_year < cntry_data["Year"].max()-5 and cntry_data["Gas_Peak_MW"].max()>1000:
            index_gas_cap_decline = cntry_data[cntry_data["Year"]>=max_gas_cap_year].index
            combined_data_new.loc[index_gas_cap_decline,"year_since_peak_gas_cap"] = combined_data_new.loc[index_gas_cap_decline,"Year"]-max_gas_cap_year
            combined_data_stats.loc[cntry,"max_gas_cap_year"] = max_gas_cap_year
            combined_data_stats.loc[cntry,"max_gas_cap"] = max_gas_cap
            
combined_data_new["Coal_Gen_Decline_Perc"] = combined_data_new["Coal_Gen_Decline_Perc"].fillna(0)
combined_data_new["Gas_Gen_Decline_Perc"] = combined_data_new["Gas_Gen_Decline_Perc"].fillna(0)
combined_data_stats.to_csv("metric_summary.csv")
#combined_data_new = combined_data_new[combined_data_new["Code"]]
combined_data_new.set_index("Country Code").to_csv("hist_coalgas_generation.csv")
combined_data_new

,Code,Year,Electricity from solar (TWh),Electricity from wind (TWh),Electricity from nuclear (TWh),Electricity from gas (TWh),Electricity from coal (TWh),Entity,Electricity generation (TWh),AR6_R10,...,Solar_FuelSwitch,Solar_Substitution,Coal_Phaseout_MW,Coal_Gen_Decline_TWh,Coal_Gen_Decline_Perc,year_since_peak_coal_gen,year_since_peak_coal_cap,Gas_Gen_Decline_TWh,Gas_Gen_Decline_Perc,year_since_peak_gas_gen
0,AFG,2000,0.00,0.0,0.0,0.0,0.00,Afghanistan,0.47,R10INDIA+,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN
1,AFG,2000,0.00,0.0,0.0,0.0,0.00,Afghanistan,0.47,R10REST_ASIA,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN
2,AFG,2001,0.00,0.0,0.0,0.0,0.00,Afghanistan,0.59,R10INDIA+,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN
3,AFG,2001,0.00,0.0,0.0,0.0,0.00,Afghanistan,0.59,R10REST_ASIA,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN
4,AFG,2002,0.00,0.0,0.0,0.0,0.00,Afghanistan,0.69,R10INDIA+,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5907,ZWE,2017,0.01,0.0,0.0,0.0,3.02,Zimbabwe,7.35,R10AFRICA,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN
5908,ZWE,2018,0.02,0.0,0.0,0.0,3.69,Zimbabwe,9.19,R10AFRICA,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN
5909,ZWE,2019,0.03,0.0,0.0,0.0,3.62,Zimbabwe,8.24,R10AFRICA,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN
5910,ZWE,2020,0.03,0.0,0.0,0.0,3.36,Zimbabwe,7.59,R10AFRICA,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN


In [7]:
print(len(combined_data_stats[combined_data_stats["max_coal_gen"].isna()==False]))
print("countries reduced coal")

32
countries reduced coal


In [8]:
print(len(combined_data_stats[combined_data_stats["max_gas_gen"].isna()==False]))
print("countries reduced gas")

21
countries reduced gas


### plus rates of decline

In [9]:
coal_gen_cntries = list(combined_data_stats[combined_data_stats["max_coal_gen"].isna()==False].index)
coal_cap_cntries = list(combined_data_stats[combined_data_stats["max_coal_cap"].isna()==False].index)
coal_both_cntries = coal_gen_cntries + list(set(coal_cap_cntries) - set(coal_gen_cntries))
hist_coal = combined_data_new[combined_data_new["Code"].isin(coal_both_cntries)]
for cntry in hist_coal["Code"].unique():
    hist_coal_cntry = hist_coal.query("Code==@cntry")
    # coal generation and coal share DECLINE rates
    coal_index = hist_coal_cntry.index
    hist_coal.loc[coal_index,"rate_coal_gen_tot"] = hist_coal.loc[coal_index,"Coal_Gen_Decline_Perc"]/hist_coal.loc[coal_index,"year_since_peak_coal_gen"]
    hist_coal.loc[coal_index,"rate_coal_cap_tot"] = hist_coal.loc[coal_index,"Coal_Share_PhasedOut"]/hist_coal.loc[coal_index,"year_since_peak_coal_cap"]

    for i in np.arange(len(coal_index))[2:-2]:
        hist_coal.loc[coal_index[i],"rate_coal_gen_5y"] = (hist_coal.loc[coal_index[i+2],"Coal_Gen_Decline_Perc"]-hist_coal.loc[coal_index[i-2],"Coal_Gen_Decline_Perc"])/5
        hist_coal.loc[coal_index[i],"rate_coal_share_5y"] = -(hist_coal.loc[coal_index[i+2],"Share_Coal_Gen"]-hist_coal.loc[coal_index[i-2],"Share_Coal_Gen"])/5*100
        hist_coal.loc[coal_index[i],"rate_coal_cap_5y"] = (hist_coal.loc[coal_index[i+2],"Coal_Share_PhasedOut"]-hist_coal.loc[coal_index[i-2],"Coal_Share_PhasedOut"])/5
    
hist_coal.set_index("Code").to_csv("hist_coal_generation.csv")
hist_coal

,Code,Year,Electricity from solar (TWh),Electricity from wind (TWh),Electricity from nuclear (TWh),Electricity from gas (TWh),Electricity from coal (TWh),Entity,Electricity generation (TWh),AR6_R10,...,year_since_peak_coal_gen,year_since_peak_coal_cap,Gas_Gen_Decline_TWh,Gas_Gen_Decline_Perc,year_since_peak_gas_gen,rate_coal_gen_tot,rate_coal_cap_tot,rate_coal_gen_5y,rate_coal_share_5y,rate_coal_cap_5y
252,AUS,1990,0.000000,0.0000,NaN,12.5655,123.36300,Australia,155.6535,R10PAC_OECD,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
253,AUS,1991,0.005450,0.0000,NaN,11.2025,127.22155,Australia,157.9635,R10PAC_OECD,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
254,AUS,1992,0.012100,0.0000,NaN,11.9640,130.00240,Australia,161.4005,R10PAC_OECD,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.000000,-0.240723,0.0
255,AUS,1993,0.014600,0.0020,NaN,12.2470,133.27740,Australia,165.3825,R10PAC_OECD,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.000000,-0.092685,0.0
256,AUS,1994,0.017400,0.0055,NaN,13.5560,136.89410,Australia,170.1425,R10PAC_OECD,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.000000,-0.371401,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5698,UZB,2018,0.010000,0.0000,0.0,41.7300,10.58000,Uzbekistan,59.4300,R10REF_ECON,...,2.0,3.0,NaN,0.0,NaN,0.698975,1.173891,3.727866,0.710942,0.0
5699,UZB,2019,0.010000,0.0000,0.0,42.1900,10.22000,Uzbekistan,60.1100,R10REF_ECON,...,3.0,4.0,NaN,0.0,NaN,1.584343,0.880418,1.286114,0.298182,0.0
5700,UZB,2020,0.010000,0.0000,0.0,41.1500,8.73000,Uzbekistan,55.9400,R10REF_ECON,...,4.0,5.0,NaN,0.0,NaN,4.659832,0.704334,-0.279590,NaN,0.0
5701,UZB,2021,0.010000,0.0000,0.0,43.9700,9.12000,Uzbekistan,59.2000,R10REF_ECON,...,5.0,6.0,NaN,0.0,NaN,3.000932,0.586945,NaN,NaN,NaN


In [10]:
gas_gen_cntries = list(combined_data_stats[combined_data_stats["max_gas_gen"].isna()==False].index)
hist_gas = combined_data_new[combined_data_new["Code"].isin(gas_gen_cntries)]

for cntry in hist_gas["Code"].unique():
    hist_gas_cntry = hist_gas.query("Code==@cntry")
    # gas generation and coal share DECLINE rates
    gas_index = hist_gas_cntry.index
    hist_gas.loc[gas_index,"rate_gas_gen_tot"] = hist_gas.loc[gas_index,"Gas_Gen_Decline_Perc"]/hist_gas.loc[gas_index,"year_since_peak_gas_gen"]
    for i in np.arange(len(gas_index))[2:-2]:
        hist_gas.loc[gas_index[i],"rate_gas_gen_5y"] = (hist_gas.loc[gas_index[i+2],"Gas_Gen_Decline_Perc"]-hist_gas.loc[gas_index[i-2],"Gas_Gen_Decline_Perc"])/5
        hist_gas.loc[gas_index[i],"rate_gas_share_5y"] = -(hist_gas.loc[gas_index[i+2],"Share_Gas_Gen"]-hist_gas.loc[gas_index[i-2],"Share_Gas_Gen"])/5*100

hist_gas.set_index("Code").to_csv("hist_gas_generation.csv")
hist_gas

,Code,Year,Electricity from solar (TWh),Electricity from wind (TWh),Electricity from nuclear (TWh),Electricity from gas (TWh),Electricity from coal (TWh),Entity,Electricity generation (TWh),AR6_R10,...,Coal_Gen_Decline_TWh,Coal_Gen_Decline_Perc,year_since_peak_coal_gen,year_since_peak_coal_cap,Gas_Gen_Decline_TWh,Gas_Gen_Decline_Perc,year_since_peak_gas_gen,rate_gas_gen_tot,rate_gas_gen_5y,rate_gas_share_5y
285,AUT,1990,0.00,0.00,0.0,7.72,6.19,Austria,49.30,R10EUROPE,...,NaN,0.0,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN
286,AUT,1991,0.00,0.00,0.0,7.47,6.84,Austria,50.18,R10EUROPE,...,NaN,0.0,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN
287,AUT,1992,0.00,0.00,0.0,6.64,4.11,Austria,49.95,R10EUROPE,...,NaN,0.0,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,-0.184216
288,AUT,1993,0.00,0.00,0.0,6.79,2.98,Austria,51.37,R10EUROPE,...,NaN,0.0,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,-0.252149
289,AUT,1994,0.00,0.00,0.0,8.64,3.10,Austria,52.11,R10EUROPE,...,NaN,0.0,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,-0.850768
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5786,VNM,2018,0.10,0.49,0.0,39.92,83.85,Vietnam,209.19,R10CHINA+,...,NaN,0.0,NaN,NaN,7.19,15.262152,3.0,5.087384,4.928890,2.300185
5787,VNM,2019,5.25,0.72,0.0,42.18,111.18,Vietnam,227.42,R10CHINA+,...,NaN,0.0,NaN,NaN,4.93,10.464869,4.0,2.616217,5.833156,2.028771
5788,VNM,2020,10.86,1.07,0.0,33.95,114.76,Vietnam,235.40,R10CHINA+,...,NaN,0.0,NaN,NaN,13.16,27.934621,5.0,5.586924,5.124177,1.701409
5789,VNM,2021,25.77,2.39,0.0,26.22,114.12,Vietnam,244.77,R10CHINA+,...,NaN,0.0,NaN,NaN,20.89,44.343027,6.0,7.390504,NaN,NaN


### export data

In [11]:
### export gas and coal data 
result_database = pd.merge(generation_data,combined_data_new[["Code","Year",
                                                              'Coal_Gen_Decline_Perc', 'Gas_Gen_Decline_Perc']],on=["Code","Year"],how="left")
#result_database = result_database.fillna(0)
result_database = result_database.drop(columns=["Entity","Electricity from solar (TWh)","Electricity from wind (TWh)","Electricity from nuclear (TWh)",
                                                "Electricity from gas (TWh)","Electricity from coal (TWh)","Electricity generation (TWh)"])

result_database = pd.merge(result_database,hist_coal[["Code","Year","rate_coal_gen_5y","rate_coal_share_5y","rate_coal_cap_5y"]],
                           on = ["Code","Year"], how = "outer")
result_database = pd.merge(result_database,hist_gas[["Code","Year","rate_gas_gen_5y","rate_gas_share_5y"]],
                           on = ["Code","Year"], how = "outer")
# can't calculate the 5 year rolling rate for the final 2 years
# some countries have different final years
rate_col = ["rate_coal_gen_5y","rate_coal_share_5y","rate_coal_cap_5y","rate_gas_gen_5y","rate_gas_share_5y"]
for cntry in result_database["Code"].unique():
    result_database_cntry = result_database.query("Code==@cntry")
    max_year = result_database_cntry["Year"].max()
    prev_years_index = result_database_cntry[result_database_cntry["Year"]<max_year-1].index
    result_database.loc[prev_years_index,rate_col] = result_database.loc[prev_years_index,rate_col].fillna(0)
    
result_database.set_index(["Code","Year"]).to_csv("_generation_results_data.csv")
result_database

,Code,Year,AR6_R10,ISO,Share_Solar_Gen,Share_Wind_Gen,Share_Nuclear_Gen,Share_Gas_Gen,Share_Coal_Gen,Coal_Gen_Decline_Perc,Gas_Gen_Decline_Perc,rate_coal_gen_5y,rate_coal_share_5y,rate_coal_cap_5y,rate_gas_gen_5y,rate_gas_share_5y
0,AFG,2000,R10INDIA+,AFG,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AFG,2000,R10INDIA+,AFG,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,AFG,2000,R10REST_ASIA,AFG,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,AFG,2000,R10REST_ASIA,AFG,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,AFG,2001,R10INDIA+,AFG,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6415,ZWE,2017,R10AFRICA,ZWE,0.001361,0.0,0.0,0.0,0.410884,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6416,ZWE,2018,R10AFRICA,ZWE,0.002176,0.0,0.0,0.0,0.401523,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6417,ZWE,2019,R10AFRICA,ZWE,0.003641,0.0,0.0,0.0,0.439320,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6418,ZWE,2020,R10AFRICA,ZWE,0.003953,0.0,0.0,0.0,0.442688,0.0,0.0,NaN,NaN,NaN,NaN,NaN


In [12]:
result_database["rate_coal_gen_5y"].max()

18.214051214707812